In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.5/870.5 kB 12.2 MB/s eta 0:00:00


In [2]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/13

In [5]:
from typing_extensions import Never
import gradio as gr
import torch
from PIL import Image
import numpy as np
import cv2
from ultralytics import YOLO

# Load the YOLOv8-seg model
model = YOLO('/content/drive/My Drive/saved_models/100best.pt')  # Use 'yolov8s-seg.pt' or your custom model path

def detect_objects(image, structures_to_detect, enabled_features):
    try:
        # Convert the image to a numpy array (YOLOv8 renders on numpy arrays)
        image_np = np.array(image)

        # Perform inference
        results = model(image_np)

        # Get the detection results (bbox, masks, confidence, class index)
        detections = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes (x1, y1, x2, y2)
        confidences = results[0].boxes.conf.cpu().numpy()  # Confidence scores
        class_indices = results[0].boxes.cls.cpu().numpy()  # Class indices
        masks = results[0].masks.data.cpu().numpy() if results[0].masks is not None else None  # Segmentation masks

        # Define custom colors (BGR format)
        color_map = {
            "teeth": (0, 255, 0),          # Green for teeth
            "mental foramen": (255, 0, 0), # Red for mental foramen
            "nasal channel": (0, 0, 255)   # Blue for nasal channel
        }

        # Get class names
        class_names = model.names

        labels_html = "<div style='display: flex; flex-wrap: wrap;'>"  # Start a flex container

        # Iterate through detections
        for i, (x1, y1, x2, y2) in enumerate(detections):
            confidence = confidences[i]
            class_idx = int(class_indices[i])

            # Ensure class index is valid
            if class_idx < len(class_names):
                class_name = class_names[class_idx]

                # Only process the structures selected by the user
                if class_name in structures_to_detect:
                    color = color_map.get(class_name, (0, 0, 255))  # Default to blue if class is not found

                    # Check if bounding box is enabled
                    if "Enable Bounding Box" in enabled_features:
                        # Draw bounding boxes around detected structures
                        cv2.rectangle(image_np, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

                    # Check if masking is enabled and masks are available
                    if "Enable Masking" in enabled_features and masks is not None:
                        # Pixel segmentation mode
                        # Get the corresponding mask for this detection
                        mask = masks[i]

                        # Resize the mask to match the input image size
                        mask_resized = cv2.resize(mask, (image_np.shape[1], image_np.shape[0]), interpolation=cv2.INTER_NEAREST)

                        # Apply the mask on the image with some transparency
                        mask_resized = mask_resized.astype(np.uint8)
                        colored_mask = np.zeros_like(image_np, dtype=np.uint8)
                        colored_mask[mask_resized == 1] = color
                        image_np = cv2.addWeighted(image_np, 1, colored_mask, 0.5, 0)

                    # Create HTML for label with better styling
                    labels_html += f'<div style="display: flex; align-items: center; margin: 5px;">' \
                                   f'<div style="width: 30px; height: 30px; background-color: rgb{color}; border: 1px solid black; margin-right: 5px;"></div>' \
                                   f'<div style="margin-left: 5px; white-space: nowrap; padding: 4px; border: 1px solid #ccc; border-radius: 5px; background-color: black;">' \
                                   f'{class_name} ({confidence:.2f})</div></div>'

        labels_html += "</div>"  # Close the flex container

        # Convert the modified numpy array back to a PIL image
        output_image = Image.fromarray(image_np)

        # Prepare the output: the image and the labels
        return output_image, labels_html

    except Exception as e:
        # Print error message for debugging
        print(f"Error during inference: {str(e)}")
        return image, f"Error during processing: {str(e)}"

# Create the Gradio interface
interface = gr.Interface(
    fn=detect_objects,               # Function to execute on image upload
    inputs=[
        gr.Image(type="pil"),        # Input type is an image
        gr.CheckboxGroup(            # Checkbox group to select which structures to detect
            choices=["teeth", "mental foramen", "nasal channel"],
            label="Select Structures to Detect",
            value=["teeth", "mental foramen", "nasal channel"],  # Default to all checked
            type="value"
        ),
        gr.CheckboxGroup(            # Checkbox group for enabling bounding box and masking
            choices=["Enable Bounding Box", "Enable Masking"],
            label="Select Features",
            value=["Enable Masking"],  # Default to all checked
            type="value"
        )
    ],
    outputs=[
        gr.Image(type="pil", label="Detected Image"),  # Output the image without labels
        gr.HTML(label="Detected Labels")                # Output labels and confidence scores as HTML
    ],
    title="FINE DENTAL STRUCTURES DETECTION FROM DENTAL XRAYS",  # Title of the interface
    description="Upload a dental X-ray image and select which dental structures to detect. Choose between bounding box and masking.",
    allow_flagging='never',  # Disable flagging
)

# Launch the Gradio app
interface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://28b88506c9ec35adf1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
